In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def renameColumns(df):
    return [col.split('-', 1)[1] if '-' in col else col for col in df.columns]

def convertTime(df):
    if df["Time"].dtype == 'int64':
        df["Time"] = pd.to_datetime(df["Time"], unit='ms')    
    else:
        df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')
    df.sort_values(by=['Time'], inplace=True)
    return df

In [ ]:
def readAndProcessCSV(filename):
    df = pd.read_csv(filename)
    df.columns = renameColumns(df)
    df = convertTime(df)
    return df

In [ ]:
def isWeekend(dateObj):
    return dateObj.weekday() in [5, 6]

def isWeeekday(dateObj):
    return not isWeekend(dateObj)

#Depending on on the mask created, you can get the data from the weekend or the week
def applyMask(df, mask):
    dfWeekend = df[~mask]
    dfWeekend = dfWeekend.reset_index(drop=True)
    return dfWeekend

def eliminateNightHours(df):
    return df[(df['Time'].dt.hour >= 6) & (df['Time'].dt.hour <= 23)]

def getDataOneHour(hour, df):
    return df[(df['Time'].dt.hour > hour) & (df['Time'].dt.hour <= hour + 1)]

def getDataOneDay(date, df):
    return df[(df['Time'].dt.day == date.day) & (df['Time'].dt.month == date.month) & (df['Time'].dt.year == date.year)]

def getDataThreeDays(date, df):
    return df[(df['Time'].dt.day >= date.day) & (df['Time'].dt.day <= date.day + 2) & (df['Time'].dt.month == date.month) & (df['Time'].dt.year == date.year)]

def eliminateDayHours(df):
    return df[(df['Time'].dt.hour > 6) & (df['Time'].dt.hour < 23)]

In [ ]:
dfOctNov = readAndProcessCSV("sensorData/08_45_18.csv")
fileNames = ["08_46_27", "08_47_03", "08_48_14", "08_48_51", "08_49_32", "08_50_13", "08_50_46"]
for fileName in fileNames:
    df = readAndProcessCSV("sensorData/" + fileName + ".csv")
    dfOctNov = pd.concat([dfOctNov, df])

#Time range of dfOctNov is 2023-10-01 to 2023-11-25
#dfOctNov["Time"].dt.date

In [ ]:
weekendMask = dfOctNov['Time'].apply(isWeekend)
dfFinal = applyMask(dfOctNov, weekendMask)
dfFinal = eliminateNightHours(dfFinal)
dfFinal = dfFinal.reset_index(drop=True)

In [ ]:
def getTemperatureMean(df, sensorName, measurementName):
    columnName = sensorName + '.' + measurementName
    dfCopy = getDataThreeDays(pd.to_datetime('2023-10-16'), df)
    return dfCopy[columnName].mean()

def getTemperatureStd(df, sensorName, measurementName):
    columnName = sensorName + '.' + measurementName
    dfCopy = getDataThreeDays(pd.to_datetime('2023-10-16'), df)
    return dfCopy[columnName].std()

temperatureMean = {}
for i in range(1, 10):
    sensorName = 'eye0' + str(i)
    temperatureMean.update({sensorName: getTemperatureMean(dfFinal, sensorName, 'temperature')})
temperatureMean.update({'eye11': getTemperatureMean(dfFinal, 'eye11', 'temperature')})
temperatureMean.update({'eye10': getTemperatureMean(dfFinal, 'eye10', 'temperature')})

for i in range(1, 7):
    sensorName = 'gas0' + str(i)
    temperatureMean.update({sensorName: getTemperatureMean(dfFinal, sensorName, 'temperature')})

for i in range(1, 6):
    sensorName = 'voc0' + str(i)
    temperatureMean.update({sensorName: getTemperatureMean(dfFinal, sensorName, 'temperature')})

temperatureMean = dict(sorted(temperatureMean.items(), key=lambda item: item[1]))

# for sensorName in temperatureMean:  
#     print(sensorName, temperatureMean[sensorName])  

In [ ]:
temperatureStd = {}
for i in range(1, 10):
    sensorName = 'eye0' + str(i)
    temperatureStd.update({sensorName: getTemperatureStd(dfFinal, sensorName, 'temperature')})
temperatureStd.update({'eye11': getTemperatureStd(dfFinal, 'eye11', 'temperature')})
temperatureStd.update({'eye10': getTemperatureStd(dfFinal, 'eye10', 'temperature')})

for i in range(1, 7):
    sensorName = 'gas0' + str(i)
    temperatureStd.update({sensorName: getTemperatureStd(dfFinal, sensorName, 'temperature')})

for i in range(1, 6):
    sensorName = 'voc0' + str(i)
    temperatureStd.update({sensorName: getTemperatureStd(dfFinal, sensorName, 'temperature')})
    
temperatureStd = dict(sorted(temperatureStd.items(), key=lambda x: x[1]))

# for sensorName in temperatureStd:  
#     print(sensorName, temperatureStd[sensorName])  